# Understanding Aqua's Operator Flow

_donny@, 30-Apr-20_

## Overview

A library for Quantum Algorithms & Applications is more than a collection of procedural code wrapped in Python functions. It needs to provide tools to make writing algorithms simple and easy. This is the layer of modules between the circuits and algorithms, providing the language and computational primitives for Quantum algorithms research.

In Aqua, we call this layer the Operator flow. It works by unifying computation with theory through the common language of functions and operators in a way which preserves physical intuition and programming freedom. In the Operator flow, we construct functions over binary variables, manipulate those functions with operators, and evaluate properties of these functions with measurements. 

Below, we'll describe the key players in the Operator flow, and show some examples of how they can be used to construct some familiar quantum algorithms. This notebook assumes a basic familiarity with Quantum Algorithms.

**Note: The Operator flow is a large and self-referential system. Do not worry if you don't immediately feel comfortable with all of its moving parts - like any system of abstractions, it can take time and play to feel comfortable. We feel that this is a worthwhile investment, as once you begin to feel comfortable with the system, algorithm development is super fast and easy.**

### Basic Design Ideas

* Batteries Included - We prioritize quick and easy access to the tools you're most likely to reach for when prototyping.

* Physically Formal - The Operator flow was built to be mathematically formal down to its very bottom primitives, so that it may serve as a lingua franca between the theory and implementation of Quantum algorithms.

* Powerful OR Readable - We've tried to let any operation be as succinct or verbose as you want it to be, including both rich syntactic sugar and long-form interfaces.

* Fast by Default - We use thoughtfully selected defaults so that algorithms run as fast as possible at many scales, and so you only need to toggle the settings you care about. We want you to be able to trust that we're running your problem as fast as we can, and if you find a way to run it faster, you should [let us know!](https://github.com/Qiskit/qiskit-aqua/issues)

### Basic Definitions

Before getting into the details of the code, it's important to note that three mathematical concepts unpin the Operator Flow. We derive most of the inspiration for the code structure from [John Watrous's formalism](https://cs.uwaterloo.ca/~watrous/TQI/) (but do not follow it exactly), so it may be worthwhile to review Chapters I and II, which are free online, if you feel the concepts are not clicking.

1. An $n$-qubit State function is a complex function over $n$ binary variables, which we will often refer to as _$n$-qubit binary strings_. For example, the traditional quantum "zero state" is a 1-qubit state function, with a definition of $Zero(0) = 1$ and $Zero(1) = 0$. It is often convenient to think about State functions as traditional kets (i.e. $Zero = |0\rangle$), but be warned, **State functions need not be normalized.** They are just functions.

1. An $n$-qubit Operator is a linear function taking $n$-qubit state functions to $n$-qubit state functions. For example, the Pauli X Operator is defined by $X(Zero) = One$ and $X(One) = Zero$. Therefore, $X(Zero)(0) = 0$. Equivalently, an Operator can be defined as a complex function over two n-qubit binary strings, and it is sometimes convenient to picture things this way. By this definition, our Pauli X can be defined by its typical matrix elements, $X(0, 0) = 0$, $X(1, 0) = 1$, $X(0, 1) = 1$, $X(1, 1) = 0$. In Aqua, our Operators are all binary, square Operators.

1. An $n-qubit Measurement$ is a functional taking n-qubit State functions to complex values. For example, a Pauli Z Measurement can be defined by $Z_{meas}(Zero) = 1$ and $Z_{meas}(One) = -1$. Measurements are simply adjoints of State Functions, and can be imagined by quantum mechanics' _bra_ notation (i.e. $Zero_{meas} = Zero^{\dagger} = \langle 0|$). Note that $Z_{meas}$ above is equivalent to $1 - 2*(\langle Zero| + \langle One|)$.

**Throughout this tutorial, we will refer to the mathematical function that an Object represents as that Object's *underlying function*.**

**TODO is "characteristic function" better?**

Note that a full expression can be written to ways:
* As function chaining: $Z(X(Zero))(0) = 0$
* As function composition: $Zero_{meas} \circ Z \circ X \circ Zero = 0$

In the Operator flow, chaining is used via the `.eval()` method, while composition is used via `.compose()` or `@` (more on this below).

### High Level Object Structure

The Operator Flow includes two primary groups of actors: 
* Operators, objects which represent functions or functionals, all deriving from `OperatorBase`: 
    * `PrimitiveOp`s - Basic Operator building blocks, whose behavior are defined by some internal computational primitive from Terra, such as a Terra's `Pauli` (`PauliOp`), `Operator` (`MatrixOp`), or `QuantumCircuit` (`CircuitOp`).
    * `StateFn`s - A class for representing both State functions and Measurements, the behavior of which is defined by some internal primitive, e.g. a simple dict (`DictStateFn`), Terra's `Statevector` (`VectorStateFn`), a `QuantumCircuit` (`CircuitStateFn`), or an OperatorBase representing a density operator (`OperatorStateFn`).
    * `ListOp`s - Classes for representing composite Operators, State functions, and Measurements constructed from expressions of others, such as addition (`SummedOp`), composition (`ComposedOp`), tensor product (`TensoredOp`), and concatination into a list (`ListOp`).
    * `EvolvedOp` - A special case of a `PrimitiveOp` holding an `Operatorbase` as its primitive, serving as a placeholder for an evolution algorithm to convert later into an Operator approximating the exponentiation of the `OperatorBase`.
* Converters, objects which manipulate Operators, all deriving from `ConverterBase`:
    * `Expectation`s - Traverse over an Operator and replace OperatorStateFn measurements into Z-basis or matrix measurements approximating the expectation value of the measurement.
    * `Evolution`s - Traverse over an Operator and replace `EvolvedOp`s with circuits approximating the evolution.
    * `CircuitSampler` - Traverse over an Operator and replace `CircuitStateFn`s with `DictStateFn`s approximating them, perhaps sampled from a quantum device.
    * Other converters, e.g. AbelianGrouper, PauliBasisChange
    
Don't worry if this seems like a whirlwind. We'll discuss each of these in detail in the next section.

## Part I: State Functions and Measurements

The most basic unit of computation in the Operator Flow is the state function, abbreviated as StateFn, a simple complex function over binary variables. There are several StateFn instances built in for convenience, `Zero, One, Plus, Minus`, and four ways a StateFn can be defined, `DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn`.

Every `StateFn` has three properties:
* `primitive` - The data structure defining the underlying function's behavior. For `DictStateFn`, this object is a `dict`, etc.
* `coeff` - A coefficient multiplying the function, i.e. `StateFn(my_primitive, coeff=3) == StateFn(my_primitive) * 3`. Note that `coeff` can be int, float, complex or a free `Parameter` object (Terra) to be bound later.
* `is_measurement` - Whether this `StateFn` is a state function or a measurement (remember, a measurement is just the adjoint of a state function).

In [1]:
from qiskit.aqua.operators import (StateFn, Zero, One, Plus, Minus, 
                                   DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn)

In [2]:
print(Zero)
print(Zero.adjoint())

DictStateFn({'0': 1})
DictMeasurement({'0': 1})


The function this object represents, mapping a binary string to a complex value, can be accessed via the `.eval` method.

In [3]:
print(Zero.eval('0'))
print(Zero.eval('1'))
print(One.eval('1'))
print(Plus.eval('0'))
print(Minus.eval('1'))

1.0
0.0
1.0
(0.70710678118655+0j)
(-0.70710678118655+0j)


This should look familiar as something analogous to the quantum state or wavefunctions you know, $|0\rangle$, $|1\rangle$, $|+\rangle$, and $|-\rangle$, but with a key difference: these statefunctions need not be normalized. `statefunction1 + statefunction2` behaves exactly as you'd expect mathematical functions to behave, producing `statefunction3`, where `statefunction3(x) = statefunction1(x) + statefunction2(x)`.

In [4]:
print((One + One + One).eval('1'))
print((One + One + One))
print((Plus + Minus).eval('0'))
print((Plus + Minus).eval('1'))

print(~Plus @ Zero)

3.0
DictStateFn({'1': 1}) * 3.0
(1.4142135623731+0j)
0j
ComposedOp(
[CircuitMeasurement(
     ┌───┐
q_0: ┤ H ├
     └───┘
),
DictStateFn({'0': 1})])


The behavior of each StateFn type is defined internally by some data structure, which we call the primitive, and a complex coefficient. The `DictStateFn` is the simplest type, holding a dict primitive of {string: complex} value pairs. `Zero` and `One` are simply these. Any value missing from the dict is simply equal to 0.

In [5]:
print(Zero)
print(One)
print(Zero.primitive)
print(Zero.coeff)
print((Zero + Zero + Zero).coeff)
print((Zero * 2j).coeff)

DictStateFn({'0': 1})
DictStateFn({'1': 1})
{'0': 1}
1.0
3.0
2j


For simplicity, the `StateFn` class also doubles as the class for Measurement. The adjoint of a `StateFn` is simply a measurement defined by the same primitive as the state. This is conceptually identical to the idea that a bra is the adjoint of a ket, and it is convenient to think of it that way.

Just as the State function over binary variables is accessible via `.eval`, so to the Measurement functional over State functions is available via `.eval`. As we'll see below, the availability of the underlying function through `.eval` is also true for the Operators.

In [6]:
print(One.adjoint())
print(Zero.adjoint().eval(One))
print(Zero.adjoint().eval(Zero))
print(Zero.adjoint().eval(Plus))

DictMeasurement({'1': 1})
0.0
1.0
(0.70710678118655+0j)


Note that we can also perform function composition between a State function and measure using `.compose`, and `.eval` with no argument later.

In [7]:
print(Zero.adjoint().compose(One).eval())
print(Zero.adjoint().compose(Zero).eval())
print(Zero.adjoint().compose(Plus).eval())

0.0
1.0
(0.70710678118655+0j)


Nearly all arithmetic operations between StateFns are supported, including:
* `+` - addition
* `-` - subtraction, negation (scalar multiplication by -1)
* `*` - scalar multiplication
* `/` - scalar division
* `@` - composition
* `^` - tensor product or tensor power (tensor with self n times)
* `**` - composition power (compose with self n times)
* `==` - equality
* `~` - adjoint, alternating between a State Function and Measurement

This arithmetic, along with the Operators, allows quick and easy construction of many states for Quantum Algorithms. However, **one must be careful to use parentheses properly when performing arithmetic in this way.**

In [8]:
print((Zero^Plus).to_circuit_op())
# TODO look into id here

CircuitStateFn(
         ┌───┐    
q_0: ────┤ H ├────
     ┌───┴───┴───┐
q_1: ┤ circuit21 ├
     └───────────┘
)


In [9]:
print(600 * ((One^5) + (Zero^5)))
print(One^Zero^3)

DictStateFn({'11111': 1.0, '00000': 1.0}) * 600.0
DictStateFn({'101010': 1})


States can also be easily converted between primitives. Below we'll see that we can also represent State functions by vectors and Quantum circuits.

In [10]:
print(((Plus^Minus)^2).to_matrix_op())
# TODO round
print(((Plus^One)^2).to_circuit_op())
print(((Plus^One)^2).to_matrix_op().sample())

VectorStateFn(Statevector([ 0.25-6.123234e-17j, -0.25+6.123234e-17j,  0.25-6.123234e-17j,
             -0.25+6.123234e-17j, -0.25+6.123234e-17j,  0.25-6.123234e-17j,
             -0.25+6.123234e-17j,  0.25-6.123234e-17j,  0.25-6.123234e-17j,
             -0.25+6.123234e-17j,  0.25-6.123234e-17j, -0.25+6.123234e-17j,
             -0.25+6.123234e-17j,  0.25-6.123234e-17j, -0.25+6.123234e-17j,
              0.25-6.123234e-17j],
            dims=(2, 2, 2, 2)))
CircuitStateFn(
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ X ├
     ├───┤
q_3: ┤ H ├
     └───┘
)
{'1101': 0.2666015625, '1111': 0.2529296875, '0111': 0.248046875, '0101': 0.232421875}


In fact, `Plus` and `Minus` are `CircuitStateFn`s.

In [11]:
print(Plus)
print(Minus)

CircuitStateFn(
     ┌───┐
q_0: ┤ H ├
     └───┘
)
CircuitStateFn(
     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     └───┘└───┘
)


Constructing a StateFn is easy. The `StateFn` class also serves as a factory, and can take any applicable primitive in its constructor and return the correct StateFn subclass. Right now the following primitives can be passed into the constructor, listed alongside the `StateFn` subclass they produce:

* str (equal to some basis bitstring) -> DictStateFn
* dict  -> DictStateFn
* Qiskit Result object -> DictStateFn
* list -> VectorStateFn
* np.ndarray -> VectorStateFn
* Statevector -> VectorStateFn
* QuantumCircuit -> CircuitStateFn
* Instruction -> CircuitStateFn
* OperatorBase -> OperatorStateFn

In [12]:
print(StateFn({'0':1}))
print(StateFn({'0':1}) == Zero)

print(StateFn([0,1,1,0]))

from qiskit.circuit.library import RealAmplitudes
print(StateFn(RealAmplitudes(2)))


DictStateFn({'0': 1})
True
VectorStateFn(Statevector([0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j],
            dims=(2, 2)))
CircuitStateFn(
     ┌──────────┐     ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ RY(θ[0]) ├──■──┤ RY(θ[2]) ├──■──┤ RY(θ[4]) ├──■──┤ RY(θ[6]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ RY(θ[1]) ├┤ X ├┤ RY(θ[3]) ├┤ X ├┤ RY(θ[5]) ├┤ X ├┤ RY(θ[7]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘└───┘└──────────┘
)


## Part II: Operators

Operators are functions which take StateFns to StateFns. For example, the identity Operator takes any StateFn to itself. Just like State Functions and Measurements, this characteristic function of the Operator is accessible via the `.eval` method, and the behavior of the function is defined by a computational primitive. If you pass a bitstring into an Operator's `eval` function, it will simply treat it as the corresponding basis StateFn.

The basic Operators in Aqua are subclasses of `PrimitiveOp`. Just like StateFn, `PrimitiveOp` is also a factory for creating the correct type of `PrimitiveOp` for a given primitive. Right now the following primitives can be passed into the constructor, listed alongside the `PrimitiveOp` subclass they produce:

* Terra's Pauli -> PauliOp
* Instruction -> CircuitOp
* QuantumCircuit -> CircuitOp
* 2d List -> MatrixOp
* np.ndarray -> MatrixOp
* spmatrix -> MatrixOp
* Terra's quantum_info.Operator -> MatrixOp

In [13]:
from qiskit.aqua.operators import X, Y, Z, I, CX, T, H, S, PrimitiveOp

In [14]:
print(X)
print(X.eval(One))
print(X.eval(One) == Zero)

print(CX)
print(CX.eval('01'))
print(CX.eval('01').eval('11'))
print(((~One^2) @ (CX.eval('01'))).eval())

print(((H^5) @ ((CX^2)^I) @ (I^(CX^2)))**2)
print((((H^5) @ ((CX^2)^I) @ (I^(CX^2)))**2) @ (Minus^5))
print(((H^I^I)@(X^I^I)@Zero))

X
DictStateFn({'0': (1+0j)})
True
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
            dims=(2, 2)))
(1+0j)
(1+0j)
          ┌───┐┌───┐     ┌───┐┌───┐
q_0: ──■──┤ I ├┤ H ├──■──┤ I ├┤ H ├
     ┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_1: ┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     └───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_2: ──■──┤ X ├┤ H ├──■──┤ X ├┤ H ├
     ┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_3: ┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     └───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_4: ─────┤ X ├┤ H ├─────┤ X ├┤ H ├
          └───┘└───┘     └───┘└───┘
CircuitStateFn(
     ┌───┐┌───┐     ┌───┐┌───┐     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├──■──┤ I ├┤ H ├──■──┤ I ├┤ H ├
     ├───┤├───┤┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_1: ┤ X ├┤ H ├┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     ├───┤├───┤└───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_2: ┤ X ├┤ H ├──■──┤ X ├┤ H ├──■──┤ X ├┤ H ├
     ├───┤├───┤┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_3: ┤ X ├┤ H ├┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     ├───┤├───┤└───┘┌─┴─┐├───┤└

## Part III: ListOps

If you've already played around with the above, you'll notice that you can easily perform operations between `StateFns` which we may not know how to perform efficiently in general (or simply haven't implemented an efficient procedure yet). In those cases, you may receive a `ListOp` result (or sublclass thereof) from your operation. For example, if you attempt to add together a `DictStateFn` and a `CircuitStateFn`, you'll receive a `SummedOp` representing the sum of the two. This composite State function still has a working `eval`, but may need to perform a non-scalable computation under the hood, such as converting both to vectors.

In [15]:
print(Zero + Plus)
print((Zero + Plus).eval('0'))

SummedOp(
[DictStateFn({'0': 1}),
CircuitStateFn(
     ┌───┐
q_0: ┤ H ├
     └───┘
)])
(1.7071067811865501+0j)


Or for composition, you might receive a `ComposedOp`. Note that just as in addition, we can sometimes perform the composition efficiently, such as between two circuit State functions. However, when composing a Measurement with a State function, we will always receive a `ComposedOp` back to reflect the fact that the State Function has been measured.

In [16]:
print('Dict measurement with Circuit statefn:')
print(~One @ Minus)
print((~One @ Minus).eval())
print('\nCircuit measurement with Circuit statefn:')
print(~One.to_circuit_op() @ Minus)
print('\nDict measurement with vector statefn:')
print(~One @ Minus.to_matrix_op())

Dict measurement with Circuit statefn:
ComposedOp(
[DictMeasurement({'1': 1}),
CircuitStateFn(
     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     └───┘└───┘
)])
(-0.70710678118655+0j)

Circuit measurement with Circuit statefn:
ComposedOp(
[CircuitMeasurement(
     ┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘
),
DictStateFn({'0': 1})])

Dict measurement with vector statefn:
ComposedOp(
[DictMeasurement({'1': 1}),
VectorStateFn(Statevector([ 0.70710678-8.65956056e-17j, -0.70710678+8.65956056e-17j],
            dims=(2,)))])


The State Functions, Operators, or Measurements from which the `ListOp` is 
The logic for how to combine the evaluation results of the `ListOp` is contained in the `.combo_fn` property of the `ListOp`. For example, 

The base `ListOp` class is a special type of ListOp. 

In [17]:
# print((~ListOp([One, Zero, Plus, Minus]) @ ListOp([One, Zero, Plus, Minus])).reduce())

## Part IV: Converters

In [18]:
import numpy as np
from qiskit.aqua.operators import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution
from qiskit.circuit import Parameter

## Construct an H2 Hamiltonian

In [19]:
two_qubit_H2 =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)

## Evolve a Bell state by Our Hamiltonian

OpFlow fully supports parameterization, so we can use a parameter for our evolution time here. Notice that there's no "evolution time" argument in any function. OpFlow exponentiates whatever operator we tell it to, and if we choose to multiply the operator by an evolution time, $e^{iHt}$, that will be refected in our exponentiation parameters. This is not some trick to make it look like Physics - it actually works this way under the hood.

In [20]:
# Meaningless state
bell = CX @ (H^I) @ Zero
# We can also do CX @ (Plus ^ Zero)
evo_time = Parameter('θ')
wf = (evo_time*two_qubit_H2).exp_i() @ bell
trot = PauliTrotterEvolution(trotter_mode='trotter').convert(wf)
# trot = MatrixEvolution().convert(wf)
trot.to_circuit().draw(fold=1000)
# trot.to_circuit().data[0]

┌───┐┌───┐┌──────────────────┐┌───┐┌───┐     ┌───┐┌──────────────────┐┌───┐┌──────────────────┐
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.18093119*θ) ├┤ X ├┤ H ├─────┤ X ├┤ RZ(-0.0112801*θ) ├┤ X ├┤ RZ(0.39793742*θ) ├
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────────┘└─┬─┘├───┤┌───┐└─┬─┘└──────────────────┘└─┬─┘├──────────────────┤
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────────■──┤ H ├┤ I ├──■────────────────────────■──┤ RZ(-0.3979374*θ) ├
     └───┘└───┘└───┘                              └───┘└───┘                              └──────────────────┘

We can bind our parameter to the operator if we so choose, and it will recursively bind into the circuit.

In [21]:
bound = trot.bind_parameters({evo_time: .5})
bound.to_circuit().draw(fold=1000)

┌───┐┌───┐┌──────────────┐┌───┐┌───┐     ┌───┐┌──────────────┐┌───┐┌─────────────┐ 
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090466) ├┤ X ├┤ H ├─────┤ X ├┤ RZ(-0.00564) ├┤ X ├┤ RZ(0.19897) ├─
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────┘└─┬─┘├───┤┌───┐└─┬─┘└──────────────┘└─┬─┘├─────────────┴┐
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────■──┤ H ├┤ I ├──■────────────────────■──┤ RZ(-0.19897) ├
     └───┘└───┘└───┘                          └───┘└───┘                          └──────────────┘

## Now that we have a state, let's measure the energy of the state

In [22]:
h2_measurement = ~StateFn(two_qubit_H2) @ bound
expect_op = PauliExpectation().convert(h2_measurement)
exact_exp_val = expect_op.eval()
print('Exact, unscallable expectation value: {}'.format(exact_exp_val))

from qiskit import BasicAer
backend = BasicAer.get_backend('qasm_simulator')

sampled_op = CircuitSampler(backend).convert(expect_op)
expectation_val = sampled_op.eval()
print('Sampled expectation value: {}'.format(expectation_val))

Exact, unscallable expectation value: (-0.65289590398246+0j)
Sampled expectation value: (-0.6451387968945299+0j)


In [23]:
print(expect_op)

SummedOp(
[ComposedOp(
[OperatorMeasurement(AbelianSummedOp(
[0.18093119 * ZZ,
-1.0523732 * II])),
CircuitStateFn(
               ┌───┐┌───┐┌──────────────┐┌───┐┌───┐┌───┐┌──────────────┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090466) ├┤ X ├┤ H ├┤ X ├┤ RZ(-0.00564) ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────┘└─┬─┘├───┤└─┬─┘└──────────────┘└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────■──┤ H ├──■────────────────────■──»
     └───┘└───┘└───┘                          └───┘                          »
«     ┌─────────────┐ ┌───┐
«q_0: ┤ RZ(0.19897) ├─┤ H ├
«     ├─────────────┴┐├───┤
«q_1: ┤ RZ(-0.19897) ├┤ H ├
«     └──────────────┘└───┘
)]),
ComposedOp(
[OperatorMeasurement(AbelianSummedOp(
[0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ])),
CircuitStateFn(
               ┌───┐┌───┐┌──────────────┐┌───┐┌───┐┌───┐┌──────────────┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090466) ├┤ X ├┤ H ├┤ X ├┤ RZ(-0.00564) ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────┘└─┬─┘├───┤└─┬─┘└─────────

### We can just as easily take the Expectation over a _vector_ of Pauli Operators.

In [24]:
ham_list = ListOp([X^X, Y^Y, Z^Z, two_qubit_H2])
expect = PauliExpectation().convert(~StateFn(ham_list) @ bound)
np.real(expect.eval())

array([ 0.01750461,  0.01750461,  0.        , -0.6528959 ])

### We Can Even Take the Expectation of a vector of Observables over a vector of StateFns

In [25]:
np.sum([[1,2,3], [4,5,6]])

21

In [28]:
# Here we're using PauliExpectation's param argument instead of passing the bound OpVec below in case we can 
# take advantage of late binding / parameterized Qobj
params = {evo_time: [.5, 1.0, 1.5]}
ham_list = ListOp([X^X, Y^Y, Z^Z, two_qubit_H2])
evolutions = (~StateFn(ham_list) @ trot).bind_parameters(params)
expects = PauliExpectation().convert(evolutions)
# print(expects)
np.real(np.around(expects.eval(), decimals=3))

array([[ 0.018,  0.064,  0.125],
       [ 0.018,  0.064,  0.125],
       [ 0.   ,  0.   ,  0.   ],
       [-0.653, -0.649, -0.646]])

### Parameter binding also supports binding lists

In [27]:
print(trot.bind_parameters(params))

ListOp(
[CircuitStateFn(
               ┌───┐┌───┐┌──────────────┐┌───┐┌───┐┌───┐┌──────────────┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090466) ├┤ X ├┤ H ├┤ X ├┤ RZ(-0.00564) ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────┘└─┬─┘├───┤└─┬─┘└──────────────┘└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────■──┤ H ├──■────────────────────■──»
     └───┘└───┘└───┘                          └───┘                          »
«     ┌─────────────┐ 
«q_0: ┤ RZ(0.19897) ├─
«     ├─────────────┴┐
«q_1: ┤ RZ(-0.19897) ├
«     └──────────────┘
),
CircuitStateFn(
               ┌───┐┌───┐┌─────────────┐┌───┐┌───┐┌───┐┌──────────────┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.18093) ├┤ X ├┤ H ├┤ X ├┤ RZ(-0.01128) ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└─────────────┘└─┬─┘├───┤└─┬─┘└──────────────┘└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■───────────────────■──┤ H ├──■────────────────────■──»
     └───┘└───┘└───┘                         └───┘                          »
«     ┌─────────────┐ 
«q_0: ┤ RZ(0.39794) ├─
«     ├───